In [8]:
import warnings
warnings.filterwarnings('ignore')
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [175]:
# Define a function to get data given a list of tickers and clean the data
def get_tickers(filepath, start_date = "2015-01-01", end_date = "2023-12-15"):
    tickers = []
    with open(filepath) as file:
        for line in file:
            tickers.append(line.strip())
    print(tickers)
    data = yf.download(tickers, start = start_date, end = end_date).sort_index()['Close']
    clean_data = data[data.isna().sum(axis = 1) != data.shape[1]]
    clean_data = clean_data.dropna(axis = 1)
    return clean_data

In [183]:
# Define a function to get the n% least volatile stocks in a DataFrame over the past t months
def get_n_least_volatile(stock_data, date, t = 36, n = 10):
    date = pd.to_datetime(date)
    start_date = date - pd.DateOffset(months=t)
    data = stock_data.copy()[start_date:date]
    ticker_vols = dict()
    for ticker in data.columns:
        return_df = np.array(data[ticker].pct_change(periods=5))[5:]
        vol = return_df.std()
        ticker_vols[ticker] = vol
    sorted_tickers = sorted(ticker_vols.items(), key = lambda x: x[1])
    return [x[0] for x in sorted_tickers[:n]]


In [161]:
def construct_portfolio(stock_data, start_date = "2021-01-01", end_date = "2023-12-15", percentage_of_all_stocks_to_buy = 0.05):
    portfolio = pd.DataFrame(data = np.zeros([len(stock_data),len(stock_data.columns)]), index = stock_data.index, columns=stock_data.columns)
    num_stocks = int(stock_data.shape[1] // (1 / percentage_of_all_stocks_to_buy))
    weight = 1 / num_stocks
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    current_month = start_date.month - 1
    for date in stock_data[start_date : end_date].index:
        if date.month != current_month:
            stocks_in_portfolio = get_n_least_volatile(stock_data = stock_data, date = date, n = num_stocks)
            current_month = date.month
            print(f"Date: {date}. Reshuffling!")
            print(f"Least Volatile Stocks:{stocks_in_portfolio}")
        for ticker in stocks_in_portfolio:
            portfolio.loc[date, ticker] = weight
    return portfolio[start_date : end_date]
        

In [149]:
def backtest(stock_data, portfolio, start_date = "2021-01-01", end_date = "2023-12-15"):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    returns = pd.DataFrame(index = portfolio.index)
    return_df = (stock_data).pct_change().shift(-1)[start_date : end_date].dropna()
    returns["daily_returns"] = (return_df * portfolio.shift(1)).sum(axis=1)
    returns["cumulative_returns"] = (returns["daily_returns"] + 1).cumprod() - 1
    returns["cumulative_%_returns"] = returns["cumulative_returns"] * 100
    returns['overall_returns'] = return_df.sum(axis=1)/(return_df.shape[1])
    returns["overall_cumulative_returns"] = (returns["overall_returns"] + 1).cumprod() - 1
    returns["overall_cumulative_%_returns"] = returns["overall_cumulative_returns"] * 100
    return returns


In [184]:
# Run the strategy
filepath = './s&p500.txt'
start_date = "2021-01-01"
end_date = "2023-12-15"
stock_data = get_tickers(filepath)
portfolio = construct_portfolio(stock_data, start_date = start_date, end_date = end_date, percentage_of_all_stocks_to_buy = 0.5)
returns = backtest(stock_data, portfolio, start_date = start_date, end_date = end_date)
stock_data.head()

['MMM', 'MSFT', 'AAPL']
[*********************100%***********************]  3 of 3 completed
Date: 2021-01-04 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-02-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-03-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-04-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-05-03 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-06-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-07-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-08-02 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-09-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-10-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-11-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2021-12-01 00:00:00. Reshuffling!
Least Volatile Stocks:['MSFT']
Date: 2022-01-03 00:00:00. Reshuffling!
Least Volatile 

,AAPL,MMM,MSFT
Date,,,
2014-12-31,27.594999,164.320007,46.450001
2015-01-02,27.332500,164.059998,46.759998
2015-01-05,26.562500,160.360001,46.330002
2015-01-06,26.565001,158.649994,45.650002
2015-01-07,26.937500,159.800003,46.230000


In [185]:
portfolio

,AAPL,MMM,MSFT
Date,,,
2021-01-04,0.0,0.0,1.0
2021-01-05,0.0,0.0,1.0
2021-01-06,0.0,0.0,1.0
2021-01-07,0.0,0.0,1.0
2021-01-08,0.0,0.0,1.0
...,...,...,...
2023-12-08,0.0,1.0,0.0
2023-12-11,0.0,1.0,0.0
2023-12-12,0.0,1.0,0.0


In [180]:
returns

,daily_returns,cumulative_returns,cumulative_%_returns,overall_returns,overall_cumulative_returns,overall_cumulative_%_returns
Date,,,,,,
2021-01-04,0.000000,0.000000,0.000000,0.003880,0.003880,0.388036
2021-01-05,0.015212,0.015212,1.521157,-0.014793,-0.010970,-1.097013
2021-01-06,-0.025662,-0.010840,-1.084043,0.012306,0.001201,0.120106
2021-01-07,-0.018265,-0.028908,-2.890784,-0.001180,0.000019,0.001920
2021-01-08,-0.008522,-0.037184,-3.718385,-0.013823,-0.013804,-1.380431
...,...,...,...,...,...,...
2023-12-08,-0.002032,-0.398764,-39.876441,-0.007596,0.197059,19.705891
2023-12-11,-0.005816,-0.402261,-40.226136,0.003466,0.201208,20.120836
2023-12-12,0.015893,-0.392761,-39.276139,0.010853,0.214245,21.424465
